In [1]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [2]:
import numpy as np

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc

import six

from tensorflow.contrib.distributions.python.ops import bernoulli
from tensorflow.contrib.distributions.python.ops import categorical
from tensorflow.contrib.seq2seq.python.ops import decoder
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import ops
from tensorflow.python.layers import base as layers_base
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.ops import embedding_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import random_ops
from tensorflow.python.ops import tensor_array_ops
from tensorflow.python.util import nest

In [4]:
def _unstack_ta(inp):
  return tensor_array_ops.TensorArray(
      dtype=inp.dtype, size=array_ops.shape(inp)[0],
      element_shape=inp.get_shape()[1:]).unstack(inp)

In [5]:
sequence_length = [3, 4, 3, 1, 0]
batch_size = 5
max_time = 8
input_depth = 7
cell_depth = 10
output_layer_depth = 3
inputs = np.random.randn(batch_size, max_time, input_depth).astype(np.float32)

In [6]:
inputs = ops.convert_to_tensor(inputs, name="inputs")

#input_tas = nest.map_structure(_unstack_ta, inputs)
input_tas = _unstack_ta(inputs)

In [7]:
inputs.eval().shape

(5, 8, 7)

In [8]:
input_tas.read(0).eval().shape

(8, 7)

In [9]:
zero_inputs = nest.map_structure(
          lambda inp: array_ops.zeros_like(inp[0, :]), inputs)

In [10]:
zero_inputs.eval().shape

(8, 7)

In [11]:
zero_inputs = array_ops.zeros_like(inputs[0,:])

In [12]:
zero_inputs.eval().shape

(8, 7)

In [13]:
from tensorflow.contrib.rnn import core_rnn_cell
from tensorflow.contrib.seq2seq.python.ops import helper as helper_py
from tensorflow.contrib.seq2seq.python.ops import basic_decoder
from tensorflow.python.framework import constant_op
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import tensor_shape
from tensorflow.python.layers import core as layers_core
from tensorflow.python.ops import variables

cell = core_rnn_cell.LSTMCell(cell_depth)
helper = helper_py.TrainingHelper(inputs, sequence_length, time_major=False)
output_layer = layers_core.Dense(output_layer_depth, use_bias=False)
expected_output_depth = output_layer_depth

In [14]:
expected_output_depth

3

In [15]:
my_decoder = basic_decoder.BasicDecoder(
          cell=cell,
          helper=helper,
          initial_state=cell.zero_state(
              dtype=dtypes.float32, batch_size=batch_size),
          output_layer=output_layer)
output_size = my_decoder.output_size
output_dtype = my_decoder.output_dtype

In [16]:
my_decoder.output_size.rnn_output

TensorShape([Dimension(3)])

In [23]:
(first_finished, first_inputs, first_state) = my_decoder.initialize()
(step_outputs, step_state, step_next_inputs, step_finished) = my_decoder.step(constant_op.constant(0), first_inputs, first_state)
batch_size_t = my_decoder.batch_size

In [24]:
sess.run(variables.global_variables_initializer())

In [25]:
first_finished.eval()

array([False, False, False, False,  True], dtype=bool)

In [26]:
first_inputs.eval().shape

(5, 7)

In [27]:
step_finished.eval()

array([False, False, False,  True,  True], dtype=bool)

In [28]:
step_outputs.rnn_output.eval()

array([[-0.0508408 , -0.05165057,  0.06775192],
       [ 0.15479779,  0.06504635, -0.01358603],
       [ 0.18893608, -0.10834984, -0.07091646],
       [-0.13847493, -0.06481301,  0.05980979],
       [-0.06738544, -0.14471821, -0.06581087]], dtype=float32)

In [29]:
step_outputs.sample_id.eval()

array([2, 0, 0, 2, 2], dtype=int32)

In [35]:
sequence_length = tf.constant([3, 4, 3, 1, 0])
finished = tf.constant([False,False,False,True,True])
next_sequence_length = array_ops.where(
          finished,
          array_ops.fill(array_ops.shape(sequence_length), 5),
          sequence_length)

In [36]:
next_sequence_length.eval()

array([3, 4, 3, 5, 5], dtype=int32)

In [41]:
step_state[0].eval()

array([[ 0.07053454, -0.14118813,  0.06967252, -0.01219992,  0.14764686,
        -0.19405451, -0.21441029,  0.1829617 , -0.09078379, -0.10884991],
       [ 0.0242832 , -0.00136052, -0.03758856, -0.13243975, -0.23338735,
         0.08539698,  0.04786929, -0.06135842,  0.15921247,  0.09439662],
       [ 0.0501725 ,  0.45520383,  0.24422514, -0.17516899, -0.56627345,
        -0.04955576, -0.03517413, -0.23947783, -0.31373349,  0.35913882],
       [-0.16829434, -0.10553734, -0.08383075,  0.092663  ,  0.0680996 ,
        -0.16200902,  0.10617378,  0.17379127, -0.25677508, -0.05221827],
       [-0.12529978,  0.27501419,  0.24914329, -0.03914498, -0.08964641,
        -0.17109002,  0.00121063, -0.14115441, -0.36193892,  0.12809984]], dtype=float32)

In [42]:
step_state[1].eval()

array([[ 0.03925622, -0.07607003,  0.03092533, -0.00532096,  0.07237594,
        -0.09604202, -0.09543931,  0.11915037, -0.04523852, -0.06443659],
       [ 0.01283682, -0.00072909, -0.01508063, -0.07387877, -0.11938155,
         0.04311427,  0.01977026, -0.02775727,  0.06083705,  0.04874465],
       [ 0.02716571,  0.13739417,  0.14161001, -0.09073706, -0.30427158,
        -0.03624927, -0.01332007, -0.0438672 , -0.19104429,  0.17898533],
       [-0.09344256, -0.04828702, -0.04067516,  0.02854112,  0.0276465 ,
        -0.06454266,  0.04380987,  0.10812437, -0.17994417, -0.02137171],
       [-0.0654664 ,  0.09974784,  0.18501829, -0.01527235, -0.048284  ,
        -0.1130731 ,  0.00075432, -0.03956325, -0.23518381,  0.06888983]], dtype=float32)

In [44]:
step_outputs.rnn_output.eval()

array([[-0.0508408 , -0.05165057,  0.06775192],
       [ 0.15479779,  0.06504635, -0.01358603],
       [ 0.18893608, -0.10834984, -0.07091646],
       [-0.13847493, -0.06481301,  0.05980979],
       [-0.06738544, -0.14471821, -0.06581087]], dtype=float32)

In [45]:
step_outputs.sample_id.eval()

array([2, 0, 0, 2, 2], dtype=int32)

In [55]:
batch_size = 5
vocabulary_size = 7
cell_depth = vocabulary_size  # cell's logits must match vocabulary size
input_depth = 10
start_tokens = [0] * batch_size
end_token = 1

embeddings = np.random.randn(vocabulary_size, input_depth).astype(np.float32)
#cell = core_rnn_cell.LSTMCell(vocabulary_size)
cell = core_rnn_cell.LSTMCell(input_depth)
helper = helper_py.GreedyEmbeddingHelper(embeddings, start_tokens, end_token)
my_decoder = basic_decoder.BasicDecoder(
          cell=cell,
          helper=helper,
          initial_state=cell.zero_state(
              dtype=dtypes.float32, batch_size=batch_size))
output_size = my_decoder.output_size
output_dtype = my_decoder.output_dtype

In [56]:
(first_finished, first_inputs, first_state) = my_decoder.initialize()
(step_outputs, step_state, step_next_inputs, step_finished) = my_decoder.step(constant_op.constant(0), first_inputs, first_state)

In [57]:
sess.run(variables.global_variables_initializer())

In [58]:
step_outputs.rnn_output.eval().shape

(5, 10)

In [59]:
step_next_inputs

<tf.Tensor 'BasicDecoderStep_3/cond/Merge:0' shape=(5, 10) dtype=float32>